# Chapter 3: Assigning Roles (Role Prompting)

（第3章：役割を与える（ロールプロンプティング））

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## Setup

次のセットアップセルを実行して、APIキーの読み込みと `get_completion` ヘルパー関数の準備を行います。

In [ ]:
%pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

def get_completion(prompt, system=''):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt}
        ],
        system=system
    )
    return message.content[0].text

---

## Lesson

Claude は、あなたが伝えたこと以外の文脈を持たない、という話を続けます。ときには、Claude に **特定の役割（必要な背景情報を含む）を演じさせる** ことが重要になります。これは **ロールプロンプティング（role prompting）** とも呼ばれます。役割の文脈は、詳しければ詳しいほど効果的です。

**役割で Claude をプライミング（事前設定）すると、さまざまな分野で性能が向上** することがあります。文章作成、コーディング、要約など幅広く有効で、人間が『○○のつもりで考えて』と言われると助けになるのと似ています。ロールプロンプティングは、Claude の応答のスタイル・トーン・話し方も変えられます。

**Note:** ロールプロンプティングは、システムプロンプトでも User メッセージの一部としても行えます。

### Examples

次の例では、ロールプロンプティングなしの場合、スケートボードについて1文で意見を求めても、Claude は **素直で装飾のない回答** を返します。

しかし、Claude に『猫』の役割を与えると、視点が変わり、**トーン／スタイル／内容が役割に合わせて適応** します。

**Note:** 追加のテクニックとして、Claude に **想定読者（audience）** を伝える方法もあります。たとえば「You are a cat」と「you are a cat talking to a crowd of skateboarders」では、かなり違う応答になります。

まずは、システムプロンプトでロール指定をしない場合の例です。

In [ ]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT))

同じユーザー質問に対して、ロールプロンプティングを入れた例です。

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

ロールプロンプティングは、特定の文体を真似させたり、特定の声色で話させたり、回答の難易度や複雑さを調整したりするのに使えます。さらに **数学や論理のタスクが得意になる** こともあります。

たとえば次の例には明確な正解（Yes）があります。しかし、この状態の Claude は誤って『情報が足りない』と判断します（実際は足りています）。

In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT))

では、Claude を **論理問題を解くボット** としてプライミングしたらどうでしょう？ 回答はどう変わるでしょうか。

この新しい役割を与えると、Claude は正解に到達します（ただし、理由付けは必ずしも完璧ではありません）。

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Note:** このコースを通じて学ぶように、同じような結果を得るための **プロンプトエンジニアリング手法は数多く存在** します。どの手法を使うかはあなた次第です。ぜひ **試行錯誤して、自分のスタイル** を見つけてください。

上の内容を変えずにレッスンのプロンプトだけ試したい場合は、ノートブック末尾の [**Example Playground**](#example-playground) を利用してください。

---

## Exercises
- [Exercise 3.1 - Math Correction](#exercise-31---math-correction)

### Exercise 3.1 - Math Correction
場合によっては、**Claude は数学（たとえ簡単なものでも）でつまずく** ことがあります。下の例では、2行目に明らかな計算ミスがあるにもかかわらず、Claude はその解答を『正しい』と評価してしまいます。なお、段階的に見れば Claude 自体はミスに気づいているのに、全体として『誤り』だと結論づけません。

`PROMPT` と / または `SYSTEM_PROMPT` を修正して、Claude が解答を `incorrectly`（誤り）と評価するようにしてください。


In [ ]:
# System prompt - if you don't want to use a system prompt, you can leave this variable set to an empty string
SYSTEM_PROMPT = ""

# Prompt
PROMPT = """Is this equation solved correctly below?

2x - 3 = 9
2x = 6
x = 3"""

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    if "incorrect" in text or "not correct" in text.lower():
        return True
    else:
        return False

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが欲しい場合は、次のセルを実行してください！

In [ ]:
print(hints.exercise_3_1_hint)

### Congrats!

ここまでの演習をすべて解けたら、次の章に進む準備ができています。Happy prompting!

---

## Example Playground

このエリアでは、このレッスンで示したプロンプト例を自由に試し、プロンプトを調整して Claude の応答がどう変わるかを観察できます。

In [ ]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))